In [4]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
# Load the full raw training CSV
df = pd.read_csv('mentalchat16k_train.csv')
print(f"Loaded dataset with {len(df)} rows")
df.head()


Loaded dataset with 16084 rows


,instruction,input,output
0,"You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. \nThe assistant gives helpful, comprehensive, and appropriate answers to the user's questions.","I've been struggling with my mental health for a while now, and I can't seem to find a way to cope with it. I've tried visualization, positive thinking, and even medication, but nothing seems to work. I've been feeling lost and helpless, and I don't know what to do next. My mind is a whirlwind of thoughts and emotions, and I can't seem to make sense of it all. I feel like I'm drowning in a sea of confusion, and I can't seem to find my way out.","I understand that you've been dealing with a sense of confusion and chaos in your thoughts and emotions for some time now. It's been a challenging journey, and it's commendable that you've tried various approaches like visualization, positive thinking, and medication to manage your symptoms. However, it's clear that these methods haven't been effective for you. It's essential to acknowledge that mental health issues can be complex and multifaceted, and what works for one person might not work for another. Today, we're going to approach this differently. We're going to identify a specific concern related to your mental health that you'd like to work on, and we'll break it down into manageable steps. We'll define the problem as a feeling of being overwhelmed by your thoughts and emotions, and we'll set realistic goals for managing this feeling. We'll explore the root causes of this confusion, such as past traumas, current stressors, and any underlying psychological conditions. We'll also examine your thought patterns and emotional responses to these triggers, and we'll develop strategies to challenge and reframe negative thoughts. This process is called Cognitive Behavioral Therapy (CBT), and it's designed to help you face your challenges more effectively by changing the way you think and feel about them. Let's begin by discussing the specific incidents or concerns that have been contributing to your feelings of confusion and chaos. We can explore these further and come up with strategies to help you manage them."
1,"You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. \nThe assistant gives helpful, comprehensive, and appropriate answers to the user's questions.","I've been feeling overwhelmed with my caregiving responsibilities, and it's been a struggle to balance these duties with my personal relationships. I've tried to communicate my limitations to my friends and church members, but they don't seem to understand or respect my boundaries. I've been dealing with high anxiety levels, which makes it even harder for me to focus on my own needs. I've tried to take care of myself, but it feels like an insurmountable task.","Your situation is complex, and it's important to acknowledge the challenges you're facing. Balancing caregiving responsibilities with personal relationships can be a delicate dance, and it's common to encounter resistance when setting boundaries. I want to help you explore strategies for communicating your needs more effectively and setting clearer boundaries. Additionally, I see that your anxiety levels are significantly impacting your ability to focus on self-care. We can work together to identify the root causes of your anxiety and develop a plan to manage it more effectively. Let's begin by exploring the specific challenges you've encountered when setting boundaries and communicating your needs. We'll also discuss potential strategies for managing your anxiety and prioritizing self-care in your life.\n\nCounselor (continued):\n"
2,"You are a helpful mental health counselling assistant, please answer the mental health questions based on the patient's description. \nThe assistant gives helpful, comprehensive, and appropriate answers to 

In [9]:
# 2. Check and drop missing values
print("Null values before drop:")
print(df.isnull().sum())
df = df.dropna()
print(f"After dropping nulls: {len(df)} rows")

Null values before drop:
instruction     0
input          27
output          0
dtype: int64
After dropping nulls: 16057 rows


In [10]:
# 3. Check and drop duplicates (keep first occurrence)
before_dup = len(df)
df = df.drop_duplicates()
after_dup = len(df)
print(f"Dropped {before_dup - after_dup} duplicate rows")

Dropped 77 duplicate rows


In [11]:
# 4. Text cleaning function
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # remove URLs
    text = re.sub(r'@\w+', '', text)                      # remove mentions
    text = re.sub(r'#\w+', '', text)                      # remove hashtags
    text = re.sub(r'\d+', '', text)                        # remove numbers
    text = re.sub(r'[^\w\s]', '', text)                    # remove punctuation
    text = re.sub(r'\s+', ' ', text)                       # remove extra spaces
    return text


In [12]:
def combine_and_clean(row):
    instr = clean_text(row['instruction'])
    inp = clean_text(row['input']) if len(str(row['input']).strip()) > 0 else ''
    return (instr + ' ' + inp).strip()

In [13]:
df['text'] = df.apply(combine_and_clean, axis=1)

In [14]:
# Also clean the output column for 'target'
df['target'] = df['output'].apply(clean_text)

In [15]:
# Keep only needed columns
df = df[['text', 'target']]

In [16]:
# 6. Split dataset 70% train, 15% val, 15% test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [17]:
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Train size: 11186
Validation size: 2397
Test size: 2397


In [18]:
# 7. Save to CSVs
train_df.to_csv('mentalchat16k_train.csv', index=False)
val_df.to_csv('mentalchat16k_validation.csv', index=False)
test_df.to_csv('mentalchat16k_test.csv', index=False)

print("Preprocessed datasets saved to data/ folder.")

Preprocessed datasets saved to data/ folder.
